In [ ]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])
project = 'd3b-bixu/dev-wgsa'

In [ ]:
def get_VEP_REFS(db_key_list):
    ref_dict = {}
    extra_db_dict = {'cadd_indels': 'CADDv1.5-38-InDels.tsv.gz', 'cadd_snvs': 'CADDv1.5-38-whole_genome_SNVs.tsv.gz',
                     'dbnsfp': 'dbNSFP4.0a.gz', 'dbscsnv': 'dbscSNV1.1_GRCh38.txt.gz', 'phylop': 'hg38.phyloP100way.bw'}
    ref_dict['reference'] = api.files.query(project=project, names=['Homo_sapiens.GRCh38.dna.toplevel.fa.gz'])[0]
    ref_dict['cache'] = api.files.query(project=project, names=['homo_sapiens_merged_vep_99_GRCh38.tar.gz'])[0]
    ref_dict['tool'] = 'VEP99'
    # db_key list has optional databases to run
    for key in 
    